In [169]:
# Pandas for data management
import pandas as pd
import numpy as np

# general imports
import datetime

# my own modules
#from scripts.pimatic_api import smartmeter

# os methods for manipulating paths
from os.path import dirname, join, abspath

from bokeh.plotting import figure
from bokeh.io import output_notebook, show

output_notebook()

Loading BokehJS ...

In [170]:
import sys
sys.path.append('../')

from pimatic_api import smartmeter

In [171]:
# Read data into dataframe
try:
    df = pd.read_csv(join('data', 'smartmeter_output.csv'), 
	                                          index_col=0)
    latest = max(pd.to_datetime(df['time'], unit='ms'))
    now = datetime.datetime.today()
    try:
        if (latest < now):
            df_new = smartmeter(start_date=latest, end_date=now)
            df = df.append(df_new, ignore_index=True)
    except:
        print('Error, geen nieuwe data opgehaald.')
except:
    now = datetime.datetime.today()
    df = smartmeter(start_date=datetime.datetime(2019, 5, 8), end_date=now)
    
df.to_csv(join('data', 'smartmeter_output.csv'))

Error, geen nieuwe data opgehaald.


In [172]:
def make_dataset(df, usageType):
        
        df_verwerkt = df[df['attributeName'] == usageType]
        df_verwerkt = df_verwerkt.sort_values('time')
        df_verwerkt = df_verwerkt.replace(0,np.NaN)
        df_verwerkt = df_verwerkt.fillna(method='ffill')
        df_verwerkt['time'] = pd.to_datetime(df_verwerkt['time'], unit='ms')
        df_verwerkt['hour'] = df_verwerkt['time'].dt.hour
        df_verwerkt['dayofweek'] = df_verwerkt['time'].dt.dayofweek
        df_verwerkt['difference'] = df_verwerkt['value'].diff()
            
        hist, edges = np.histogram(df_verwerkt['difference'], density=True, bins=50)
        return hist, edges
   
def make_plot(title, hist, edges):

    p = figure(title=title, tools='', background_fill_color="#fafafa")
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
       fill_color="navy", line_color="white", alpha=0.5)

    p.y_range.start = 0
    p.legend.location = "center_right"
    p.legend.background_fill_color = "#fefefe"
    p.xaxis.axis_label = 'x'
    p.yaxis.axis_label = 'Pr(x)'
    p.grid.grid_line_color="white"
    return p

In [173]:
hist, edges = make_dataset(df, 'gastotalusage')

/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:32: RuntimeWarning: invalid value encountered in reduce
  return umr_minimum(a, axis, None, out, keepdims, initial)
/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:28: RuntimeWarning: invalid value encountered in reduce
  return umr_maximum(a, axis, None, out, keepdims, initial)


ValueError: autodetected range of [nan, nan] is not finite

In [174]:
p = make_plot('test', hist, edges)

In [175]:
import os
os.path.abspath('')

'/Users/wouternieuwerth/Desktop/Jupyter/20190316_PimaticAPI_Spyder/bokeh_app'

In [176]:
cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory
print("Files in '%s': %s" % (cwd, files))

Files in '/Users/wouternieuwerth/Desktop/Jupyter/20190316_PimaticAPI_Spyder/bokeh_app': ['.DS_Store', 'testing.ipynb', 'scripts', '.ipynb_checkpoints', 'main.py', 'data']


In [177]:
df_verwerkt = df[df['attributeName'] == 'gastotalusage']
df_verwerkt = df_verwerkt.sort_values('time')
df_verwerkt = df_verwerkt.replace(0,np.NaN)
df_verwerkt = df_verwerkt.fillna(method='ffill')
df_verwerkt = df_verwerkt.fillna(method='bfill') # nodig om eerste rij te fixen.
df_verwerkt['time'] = pd.to_datetime(df_verwerkt['time'], unit='ms')
df_verwerkt['hour'] = df_verwerkt['time'].dt.hour
df_verwerkt['minutes'] = (df_verwerkt['time'].dt.hour * 60) + df_verwerkt['time'].dt.minute
df_verwerkt['dayofweek'] = df_verwerkt['time'].dt.dayofweek
df_verwerkt['difference'] = df_verwerkt['value'].diff()
df_verwerkt['difference'] = df_verwerkt['difference'].fillna(0)

pivot = pd.pivot_table(df_verwerkt, index='dayofweek', columns='minutes', values='difference', aggfunc=sum)
pivot = pivot.fillna(0)
stacked = pivot.stack().reset_index()
stacked = stacked.rename(columns={0:'aantal'})

In [178]:
df_verwerkt['difference'].describe()

count    72064.000000
mean         0.010363
std          0.028809
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.238000
Name: difference, dtype: float64

In [179]:
pivot.head()

minutes,0,1,2,3,4,5,6,7,8,9,...,1430,1431,1432,1433,1434,1435,1436,1437,1438,1439
dayofweek,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.003,0.0,0.0,0.0,0.000,0.01,0.0,0.0,0.0,...,0.013,0.000,0.013,0.018,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.000,0.0,0.0,0.0,0.013,0.00,0.0,0.0,0.0,...,0.000,0.012,0.018,0.000,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000,0.0,0.0,0.0,0.013,0.00,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.000,0.0,0.0,0.0,0.000,0.00,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000,0.0,0.0,0.0,0.000,0.00,0.0,0.0,0.0,...,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.0,0.0


In [180]:
stacked.head()

,dayofweek,minutes,aantal
0,0,0,0.000
1,0,1,0.003
2,0,2,0.000
3,0,3,0.000
4,0,4,0.000


In [181]:
df_verwerkt['difference'].head()

1145    0.0
1141    0.0
1137    0.0
1133    0.0
1129    0.0
Name: difference, dtype: float64

In [182]:
hist, edges = np.histogram(pivot, density=True, bins=50)

In [183]:
p = make_plot('test', hist, edges)

In [184]:
show(p)

In [196]:
palette = inferno(25)
color_mapper = LinearColorMapper(palette=palette)

p2 = figure()
p2.vbar(x=stacked['minutes'], bottom=0, top=stacked['aantal'], width=0.9)
ticks = pd.date_range('2018-01-01', periods=24, freq='H').tolist()
p2.xaxis.ticker = FixedTicker(ticks=ticks)
show(p2)

ValueError: expected an element of Seq(Float), got seq with invalid items [Timestamp('2018-01-01 00:00:00', freq='H'), Timestamp('2018-01-01 01:00:00', freq='H'), Timestamp('2018-01-01 02:00:00', freq='H'), Timestamp('2018-01-01 03:00:00', freq='H'), Timestamp('2018-01-01 04:00:00', freq='H'), Timestamp('2018-01-01 05:00:00', freq='H'), Timestamp('2018-01-01 06:00:00', freq='H'), Timestamp('2018-01-01 07:00:00', freq='H'), Timestamp('2018-01-01 08:00:00', freq='H'), Timestamp('2018-01-01 09:00:00', freq='H'), Timestamp('2018-01-01 10:00:00', freq='H'), Timestamp('2018-01-01 11:00:00', freq='H'), Timestamp('2018-01-01 12:00:00', freq='H'), Timestamp('2018-01-01 13:00:00', freq='H'), Timestamp('2018-01-01 14:00:00', freq='H'), Timestamp('2018-01-01 15:00:00', freq='H'), Timestamp('2018-01-01 16:00:00', freq='H'), Timestamp('2018-01-01 17:00:00', freq='H'), Timestamp('2018-01-01 18:00:00', freq='H'), Timestamp('2018-01-01 19:00:00', freq='H'), Timestamp('2018-01-01 20:00:00', freq='H'), T

In [130]:
from bokeh.models import ColumnDataSource, LinearColorMapper
from bokeh.palettes import inferno
from bokeh.models.tickers import FixedTicker

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

palette = inferno(25)
color_mapper = LinearColorMapper(palette=palette)

def make_dataset(df, usageType):
        
    df_verwerkt = df[df['attributeName'] == 'gastotalusage']
    df_verwerkt = df_verwerkt.sort_values('time')
    df_verwerkt = df_verwerkt.replace(0,np.NaN)
    df_verwerkt = df_verwerkt.fillna(method='ffill')
    df_verwerkt = df_verwerkt.fillna(method='bfill') # nodig om eerste rij te fixen.
    df_verwerkt['time'] = pd.to_datetime(df_verwerkt['time'], unit='ms')
    df_verwerkt['hour'] = df_verwerkt['time'].dt.hour
    df_verwerkt['minutes'] = (df_verwerkt['time'].dt.hour * 60) + df_verwerkt['time'].dt.minute
    df_verwerkt['dayofweek'] = df_verwerkt['time'].dt.dayofweek
    df_verwerkt['difference'] = df_verwerkt['value'].diff()
    df_verwerkt['difference'] = df_verwerkt['difference'].fillna(0)
    
    groupby = df_verwerkt.groupby('hour').sum()

    return ColumnDataSource(data=groupby)

def make_plot(src):

    p = figure(title='Bar chart', tools=TOOLS, background_fill_color="#fafafa", plot_width=900, plot_height=400, x_range=(-0.5,23.5))
    p.vbar(x='hour', bottom=0, top='difference', source=src, width=1, fill_color={'field': 'difference', 'transform': color_mapper},)

    p.y_range.start = 0
    p.legend.location = "center_right"
    p.legend.background_fill_color = "#fefefe"
    p.xaxis.axis_label = 'Uur van de dag'
    p.yaxis.axis_label = 'Verbruik'
    p.grid.grid_line_color="white"
    p.xaxis.ticker = FixedTicker(ticks=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23])
    return p

def update(attr, old, new):

    # Get the selected items for the graph
    # ...
    print('update')



# initial execution
src = make_dataset(df, 'gastotalusage')
p = make_plot(src)

In [131]:
show(p)

In [163]:
from bokeh.models import ColumnDataSource, LinearColorMapper
from bokeh.palettes import inferno
from bokeh.models.tickers import FixedTicker

TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

palette = inferno(25)
color_mapper = LinearColorMapper(palette=palette)

def make_dataset(df, usageType):
        
    df_verwerkt = df[df['attributeName'] == 'gastotalusage']
    df_verwerkt = df_verwerkt.sort_values('time')
    df_verwerkt = df_verwerkt.replace(0,np.NaN)
    df_verwerkt = df_verwerkt.fillna(method='ffill')
    df_verwerkt = df_verwerkt.fillna(method='bfill') # nodig om eerste rij te fixen.
    df_verwerkt['time'] = pd.to_datetime(df_verwerkt['time'], unit='ms')
    df_verwerkt['hour'] = df_verwerkt['time'].dt.hour
    df_verwerkt['minutes'] = (df_verwerkt['time'].dt.hour * 60) + df_verwerkt['time'].dt.minute
    df_verwerkt['dayofweek'] = df_verwerkt['time'].dt.dayofweek
    df_verwerkt['difference'] = df_verwerkt['value'].diff()
    df_verwerkt['difference'] = df_verwerkt['difference'].fillna(0)
    
    groupbyVertical = df_verwerkt.groupby('dayofweek').sum()

    return ColumnDataSource(data=groupbyVertical)

def make_plot(src):

    p = figure(title='Bar chart', tools=TOOLS, background_fill_color="#ffffff", plot_width=400, plot_height=400, y_range=(-0.5,6.5), y_axis_location="right", tooltips=[('Verbruik', '@difference'),('Dag','@dayofweek')])
    p.hbar(y='dayofweek', left=0, right='difference', source=src, height=0.96, color={'field': 'difference', 'transform': color_mapper})

    p.xaxis.axis_label = 'Verbruik'
    p.yaxis.axis_label = 'Dag van de week'
    p.grid.grid_line_color="#ffffff"
    p.yaxis.ticker = FixedTicker(ticks=[0,1,2,3,4,5,6])
    p.yaxis.major_label_overrides = {0:'Maandag', 1:'Dinsdag', 2:'Woensdag', 3:'Donderdag', 4:'Vrijdag', 5:'Zaterdag', 6:'Zondag'}
    return p

def update(attr, old, new):

    # Get the selected items for the graph
    # ...
    print('update')



# initial execution
src = make_dataset(df, 'gastotalusage')
p2 = make_plot(src)

In [164]:
show(p2)